I use this jupyter to test whether my RNN is correct or not (Mar 20, 2023).



## Import pkgs

In [1]:
import sys
sys.path.append("../mypkg")
from constants import RES_ROOT, FIG_ROOT, DATA_ROOT

In [2]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
from easydict import EasyDict as edict
from tqdm import trange

plt.style.use(FIG_ROOT/"base.mplstyle")
%matplotlib inline

In [3]:
# pkgs for pytorch (Mar 20, 2023)
import torch
import torch.nn as nn

torch.set_default_dtype(torch.float64)
torch.set_default_tensor_type(torch.DoubleTensor)

## Data, fns and parameters

In [4]:
paras = edict()
paras.d = 68 # dim of feature vector
paras.n = 1000 # length of the time series

paras.n_ouput = 7 # dim of output from RNN
paras.n_hidden = 128 # dim of hidden from RNN

In [5]:
# generate a toy data
torch.manual_seed(0)
Amat = torch.randn(paras.d, paras.n_ouput)
tmp_data_x = torch.tensor(np.random.randn(paras.d, paras.n))
tmp_data_y = (tmp_data_x.T @ Amat).T

In [6]:

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input_, hidden):
        combined = torch.cat((input_, hidden), 1)
        output = self.i2o(combined)
        hidden = self.i2h(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)


In [7]:
# functions to generate training sample
# note that I fix the bach size = 1
def random_choice(n):
    """Randomly select the lower and upper bound of the segment
        args:
            n: len of the total time series
    """
    up_bd = torch.randint(low=100, high=n, size=(1, ))
    len_seg = torch.randint(low=10, high=50, size=(1, ))
    low_bd = up_bd - len_seg
    return low_bd.item(), up_bd.item()


def random_training_samples(data_x, data_y):
    """Randomly select a sample from the whole segment
        args:
            data_x: total ts
            data_y: total label ts for data_x
    """
    low_bd, up_bd = random_choice(paras.n)
    part_data_x = torch.unsqueeze(data_x[:, low_bd:up_bd], 0)
    part_data_x = part_data_x.permute(2, 0, 1)
    
    part_data_y = torch.unsqueeze(data_y[:, low_bd:up_bd], 0)
    part_data_y = part_data_y.permute(2, 0, 1)
    return part_data_x, part_data_y

In [8]:
criterion = nn.MSELoss()
learning_rate = 0.005 # If you set this too high, it might explode. If too low, it might not learn

def train(data_x, data_y):
    hidden = rnn.initHidden()

    rnn.zero_grad()

    loss = 0
    seg_len = data_x.shape[0]
    for i in range(seg_len):
        output, hidden = rnn(data_x[i], hidden)
        loss = criterion(output, data_y[i])/seg_len + loss

        
    loss.backward()
    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        p.data.add_(p.grad.data, alpha=-learning_rate)

    return output, loss.item()

In [12]:
rnn = RNN(paras.d, paras.n_hidden, paras.n_ouput)

In [13]:
import time
import math

n_iters = 1000000
print_every = 500
plot_every = 100



# Keep track of losses for plotting
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

for iter in range(1, n_iters + 1):
    data_x, data_y = random_training_samples(tmp_data_x, tmp_data_y)
    ouput, loss = train(data_x, data_y)
    current_loss += loss

    # Print iter number, loss, name and guess
    if iter % print_every == 0:
        print('%d %d%% (%s) %.4f' % (iter, iter / n_iters * 100, timeSince(start), loss))

    # Add current loss avg to list of losses
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

500 0% (0m 3s) 59.9952
1000 0% (0m 6s) 48.2808
1500 0% (0m 10s) 38.8679
2000 0% (0m 15s) 42.7155
2500 0% (0m 19s) 49.9927
3000 0% (0m 22s) 52.8567
3500 0% (0m 26s) 55.9514
4000 0% (0m 29s) 37.4992
4500 0% (0m 32s) 51.1702
5000 0% (0m 36s) 39.5328
5500 0% (0m 39s) 47.2552
6000 0% (0m 43s) 53.4964
6500 0% (0m 46s) 47.7844
7000 0% (0m 49s) 46.8868
7500 0% (0m 53s) 37.0808
8000 0% (0m 56s) 32.5652
8500 0% (0m 59s) 52.7252
9000 0% (1m 3s) 41.7861
9500 0% (1m 6s) 40.7708
10000 1% (1m 9s) 55.1898
10500 1% (1m 13s) 42.9478
11000 1% (1m 16s) 54.0115
11500 1% (1m 19s) 39.1690
12000 1% (1m 23s) 58.2272
12500 1% (1m 26s) 58.9865
13000 1% (1m 29s) 53.7657
13500 1% (1m 33s) 58.3969
14000 1% (1m 36s) 40.7940
14500 1% (1m 39s) 35.3505
15000 1% (1m 43s) 36.1115
15500 1% (1m 46s) 52.8059
16000 1% (1m 50s) 51.2405


KeyboardInterrupt: 